In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: {e}")

✅ Gemini API key setup complete.


In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

In [5]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, google_search
from google.genai import types

# Retry config
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Research Agent: Finds information
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized research agent. 
    Use google_search to find 2–3 relevant pieces of information on the topic 
    and present them with citations.""",
    tools=[google_search],
    output_key="research_findings",
)

# Summarizer Agent: Summarizes findings
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Read the research findings: {research_findings}
    Create a concise summary as a bulleted list of 3–5 key points.""",
    output_key="final_summary",
)

# Root Agent: Coordinates workflow
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a research coordinator. 
    1. Call ResearchAgent to gather information. 
    2. Then call SummarizerAgent to create a summary. 
    3. Finally, present the summary to the user.""",
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ All agents created successfully.")


✅ All agents created successfully.


In [6]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("What are the latest advancements in quantum computing and what do they mean for AI?")



 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


In [7]:
response


[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'request': 'latest advancements in quantum computing and their implications for AI'
         },
         id='adk-ba254f5b-821b-4141-99e3-6b7df683850d',
         name='ResearchAgent'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=23,
   prompt_token_count=142,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=142
     ),
   ],
   total_token_count=165
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None,

In [8]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [9]:
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a specialized research agent. Your ONLY job is to use the google_search tool to find 3 relevant and recent pieces of information 
on the topic the user provides. ALWAYS return:
1) A short bullet list of the 3 results with one-line summaries and URLs,
2) The date of each source if available.

Focus specifically on hardware: chips, GPUs, TPUs, AI ASICs, interconnects, or major architecture improvements.
""",
    tools=[google_search],
    output_key="research_findings",
)

In [10]:
print(type(response))


<class 'list'>


In [11]:
# Robust ADK response inspector for Kaggle notebooks
from datetime import datetime
import json
import warnings

# Optional: hide the repeated non-text warnings if you want
warnings.filterwarnings("ignore", message="Warning: there are non-text parts in the response.*")

def inspect_adk_response(resp):
    """
    Accepts the object returned by InMemoryRunner.run_debug (usually a list of events)
    and prints:
      - timestamp / author for each event
      - function calls
      - function responses (result if present)
      - any state_delta keys (useful for output_key results like 'final_summary')
    Works even when some parts are None or content.parts is empty.
    """
    # If it's not a list, try to normalize
    events = resp if isinstance(resp, list) else [resp]

    last_texts = []
    print("\n=== ADK Event Timeline ===\n")

    for ev in events:
        # Basic metadata
        author = getattr(ev, "author", None)
        ts = getattr(ev, "timestamp", None)
        ts_str = datetime.fromtimestamp(ts).isoformat() if ts else "unknown-time"
        print(f"--- Event by {author or 'UNKNOWN'} at {ts_str} ---")

        # 1) Inspect content.parts if present
        content = getattr(ev, "content", None)
        parts = getattr(content, "parts", None) if content else None

        if parts:
            for i, part in enumerate(parts, start=1):
                print(f"  -- part {i} --")
                # function_call
                fc = getattr(part, "function_call", None)
                if fc:
                    # Print name and args safely
                    name = getattr(fc, "name", None)
                    args = getattr(fc, "args", None)
                    print(f"    [function_call] name={name} args={json.dumps(args, default=str)}")

                # function_response
                fr = getattr(part, "function_response", None)
                if fr and getattr(fr, "response", None):
                    resp_obj = fr.response
                    # If response contains a "result" string, print it
                    if isinstance(resp_obj, dict) and "result" in resp_obj:
                        print("\n    [function_response.result]\n", resp_obj["result"].strip(), "\n")
                        last_texts.append(resp_obj["result"].strip())
                    else:
                        print("    [function_response]", json.dumps(resp_obj, indent=2, default=str))

                # plain text part
                txt = getattr(part, "text", None)
                if txt:
                    print("    [text]\n", txt.strip())
                    last_texts.append(txt.strip())

        else:
            print("  -- content.parts empty or missing --")

        # 2) Inspect actions.state_delta (common place for output_key values)
        actions = getattr(ev, "actions", None)
        state_delta = getattr(actions, "state_delta", None) if actions else None
        if state_delta:
            try:
                # state_delta might be a dict-like object
                sd = dict(state_delta)
            except Exception:
                sd = state_delta
            print("  [state_delta keys]:", ", ".join(sd.keys()) if hasattr(sd, "keys") else str(sd))
            # Print each key's value (truncate long text lightly)
            for k, v in (sd.items() if hasattr(sd, "items") else []):
                # v might be a long string
                if isinstance(v, str) and len(v) > 1500:
                    print(f"    {k}: (long text, length={len(v)})\n    {v[:1000]}...\n")
                else:
                    print(f"    {k}: {v}\n")
                # collect final_summary or similarly named keys for final display
                if isinstance(v, str):
                    last_texts.append(v.strip())

        print()  # blank line between events

    # Print a combined final output if we collected any textual pieces
    if last_texts:
        print("\n=== Combined Extracted Text (most relevant pieces) ===\n")
        # De-duplicate near-identical entries while preserving order
        seen = set()
        combined = []
        for t in last_texts:
            snippet = t.strip()
            if snippet and snippet not in seen:
                seen.add(snippet)
                combined.append(snippet)
        print("\n\n".join(combined[:10]))  # print up to first 10 pieces
    else:
        print("⚠️ No textual outputs found in the events.")

# Example usage (run right after your run_debug call)
# runner = InMemoryRunner(agent=root_agent)
# response = await runner.run_debug("Summarize the latest developments in AI hardware accelerators.")
# inspect_adk_response(response)


In [12]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Summarize the latest developments in AI hardware accelerators.")


 ### Created new session: debug_session_id

User > Summarize the latest developments in AI hardware accelerators.


ResearchCoordinator > The latest developments in AI hardware accelerators are characterized by a strong push for increased performance and efficiency through specialized chips, including ASICs like Google's TPUs. AI is also being integrated into the chip design process itself, leading to faster and more cost-effective development. There's a significant trend towards edge AI hardware for on-device processing and an accelerated annual release cycle driven by intense competition among established players and emerging startups.


In [13]:
pretty_show_adk_response(response)


NameError: name 'pretty_show_adk_response' is not defined